In [ ]:
import pandas as pd
import re
import numpy as np
from tqdm import tqdm

In [ ]:
df_train = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv",nrows=10000)

In [ ]:
def get_text(filename, test=False):
    if test:
        df = pd.read_json('../input/coleridgeinitiative-show-us-the-data/test/{}.json'.format(filename))
    else:
        df = pd.read_json('../input/coleridgeinitiative-show-us-the-data/train/{}.json'.format(filename))
    text = " ".join(list(df['text']))
    return text

df_train['text'] = df_train['Id'].apply(get_text)

In [ ]:
# text = 'I created this project on datasets indian families of my datasets of comon datasets indian families'
# entity = 'datasets indian families'

In [ ]:
def convert_tokens(x,m,train=True):
    df = pd.DataFrame()
    text = x['text'].replace('\uf0b7','').split()
    entity = x['dataset_label']
    
    ##
    tokens=[]
    k=0
    for i,x in enumerate(text):

        if k==0:
            if x==entity.split()[0]:
                entity_len = len(entity.split())
                if entity == ' '.join(text[i:i+entity_len]):
                    tokens.extend(['o-dataset']*len(entity.split()))
                    k = entity_len
                    #print('k updated')
                else:
                    #print(x,'o1')
                    tokens.append('o')
            else:
                #print(x,'o2')
                tokens.append('o')


        k = max(0,k-1)
            
    k=0
    sentence_hash=[]
    for i in range(0,len(text),290):
        sentence_hash.extend([f'sentence#{k}']* len(text[i:i+290]))
        k+=1
      
    df['token']=list(map(str,text))
    df['sentence#'] = sentence_hash
    df['sentence'] = f'sentence{m}'
    df['entity'] = tokens
    
#     if train:
#         def decide(x):
#             for k in label:
#                 if x.find(k)!=-1:
#                     return 'o-dataset'

#             return 'o'
#         label = x['dataset_label'].split()
#         df['entity'] = df['token'].map(lambda x : decide(x))    
    
    return df

In [ ]:
final = pd.DataFrame()
for i,row in tqdm(df_train.iterrows()):
    df = convert_tokens(row,i)
    final = final.append(df,ignore_index=True)

##making some correction of bad tagging    
# masks = (final['entity'].shift()==final['entity'])|(final['entity']==final['entity'].shift(-1))    
# final.loc[~masks,'entity'] = 'o'

In [ ]:
final.to_pickle("show_us_the_data_train_ner.pkl")
